In [1]:
import sys
sys.path.extend(['../src/'])
from config import *
config = Config()

from pyhanlp import *

f = open(config.userdict, encoding = 'UTF-8')

ner_pos = {}
for line in f:
    res = line.strip().split('	')
    word = res[0]
    nature = res[1]
    ner_pos[word] = nature

f = open('../source/data/ner_data/synonym.txt', 'w+', encoding = 'UTF-8')
CoreSynonymDictionary = JClass("com.hankcs.hanlp.dictionary.CoreSynonymDictionary")
word_array = [i for i in ner_pos.keys()]
similarity = {}
for a in word_array:
    similarity[a] = []
    for b in word_array[word_array.index(a)+ 1:]:
        if CoreSynonymDictionary.similarity(a, b) == 1.0:
            f.write(a + '\t' + b + '\n')
f.close()

In [2]:
from pyhanlp import *

f = open(config.userdict, encoding = 'UTF-8')

ner_pos = {}
for line in f:
    res = line.strip().split('	')
    word = res[0]
    nature = res[1]
    ner_pos[word] = nature

In [3]:
config.label_dict = {'检查和检验': 'CHECK',
              '症状和体征': 'SIGNS',
              '疾病和诊断': 'DISEASE',
              '治疗': 'TREATMENT',
              '身体部位': 'BODY'}

In [73]:
import re

f = open('../source/data/ner_data/ner_train.txt', 'w+',encoding = 'UTF-8')
for root,dirs,files in os.walk(config.origin_datapath):
    for file in files:
        filepath = os.path.join(root, file)    
        res_dict = {}
        text = re.sub('\n', '', open(filepath, encoding = 'UTF-8').read())
        sg_word = sorted([word for word in ner_pos.keys() if word in text],key = lambda i:len(i),reverse=False)
        sg_words = []
        for word in sg_word:
            sg_words = sg_words + [word] * text.count(word)
        sg_nature = [ner_pos[word] for word in sg_words]
        words_begin = []; words_end = []
        for word in sg_word:
            if word in ['胸部正位+左斜位片','A+B']:
                word = word[:word.index('+')] + '\\'+ word[word.index('+'):]
            words_begin = words_begin + [m.start() for m in re.finditer(word, text)]
            words_end = words_end + [m.end()-1 for m in re.finditer(word, text)]
        res_dict = {}
        for w in range(len(sg_words)):
            res = sg_words[w]
            start = words_begin[w]
            end = words_end[w]
            label = config.label_dict[sg_nature[w]]
            for i in range(start, end+1):
                if i == start:
                    label_cate = 'B-' + label
                else:
                    label_cate = 'I-' + label
                res_dict[i] = label_cate
        for indx, char in enumerate(text):
            char_label = res_dict.get(indx, 'O')
            f.write(char + '\t' + char_label + '\n')
f.close()